<a href="https://colab.research.google.com/github/YassineZITI/Django_DS_Books/blob/master/Django_recomander_system_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Content Based Recommander Sytem That will help to disply similar books to a book.</h3>
<p><a href='https://github.com/YassineZITI/IT-or-Not-Books-'>Here</a> i built all external models and data used in this notebook</p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nltk --q
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from tensorflow.keras.models import load_model
import pickle
import re
import numpy as np
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda
from sklearn.metrics.pairwise import cosine_similarity
# Create your views here.

def preprocess(description):
	stop_words = set(stopwords.words('english'))
	desc = re.sub('[^A-Za-z]+',' ',description)
	desc = desc.lower()
	#remove newlines and whitelines
	desc = re.sub('\n|\r',' ',desc)
	desc = ' '.join([i for i in desc.split(' ') if i not in stop_words])
	with open('/content/drive/My Drive/tokenizer.pkl','rb') as f: 
		tokenizer = pickle.load(f)
	sequences = tokenizer.texts_to_sequences([desc])
	x = pad_sequences(sequences,maxlen=100,padding='post')
	return x
def EmbeddModel(maxlen,emb_matrix_path):
	with open(emb_matrix_path,'rb') as f:
		emb_matrix = pickle.load(f)
	model = Sequential()
	model.add(Embedding(input_dim=emb_matrix.shape[0], output_dim=emb_matrix.shape[1], input_length=maxlen , weights=[emb_matrix]))
	# Average the output of the Embedding layer over the word dimension
	model.add(Lambda(lambda x: tf.keras.backend.mean(x, axis=1)))
	return model

def predict(description):
	x = preprocess(description)
	classifier = load_model('/content/drive/My Drive/classifier.h5') 
	embedder = EmbeddModel(100,'/content/drive/My Drive/emb_matrix.pkl')
	embedding = embedder.predict(x)
	prediction = classifier.predict(x)
	it_or_not = ['IT','Not']
	prediction = it_or_not[np.argmax(prediction)]
	return prediction,embedding

def similars(embedding,embeddings_isbn,isbn):
	embed = np.array([i[1] for i in embeddings_isbn])
	indices =  np.argsort(cosine_similarity([embedding], embed))[0][::-1]
	similar = ' '.join([i[0] for i in embeddings_isbn[indices[1:9]]])
	embeddings_isbn = np.vstack((embeddings_isbn,np.array((isbn,embedding))))
	return similar,embeddings_isbn


In [ ]:
emb_matrix_path = '/content/drive/My Drive/emb_matrix.pkl'

In [ ]:
with open(emb_matrix_path,'rb') as f:
		emb_matrix = pickle.load(f)

In [ ]:
with open('/content/drive/My Drive/app_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
categories = ['Database','TensorFlow','Neural Networks','Hadoop',
              'NumPy','Visualization','Machine Learning',
              'Analytics','Apache','Big Data','Scala','Microsoft']
counts = {c:0 for c in categories}


In [ ]:
Data = []
for book in data:
  if counts[book['category']] <=50:
    Data.append(book)
    counts[book['category']]+=1
  

In [ ]:
len(Data)

584

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:

for i in range(len(Data)):
  embedding = predict(Data[i]['description'])[1]
  Data[i]['embedding'] = embedding[0]

In [ ]:
embeddings = np.array([(book['isbn13'],book['embedding']) for book in Data])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
cosine_similarity(
    [embeddings[0]],
    embeddings[1:10]
)


array([[0.89961517, 0.8904499 , 0.9000989 , 0.8729072 , 0.85745776,
        0.9420755 , 0.76154196, 0.7778647 , 0.9270598 ]], dtype=float32)

In [ ]:
import numpy as np
book = Data[1]
indices =  np.argsort(cosine_similarity([embeddings[1]], embeddings))[0][-9:]

indices[:-1]

array([ 39, 256, 479,  11, 422,   6, 378,   3])

In [ ]:
for i in range(len(Data)) :
  book = Data[i]
  indices =  np.argsort(cosine_similarity([embeddings[i]], embeddings))[0][::-1]
  
  similar = ''

  for ind in indices[1:]:
    if Data[ind]['category'] == book['category']:
      similar += Data[ind]['isbn13'] + ' '
      if len(similar) >= 112:
        break
  book['similar'] = similar
  Data[i] = book

In [ ]:
Data[300]['similar']

'9781098115784 9781788390040 9781785882951 9781788474399 9781484234310 9781788996402 9781484280164 9781784393908 '

In [ ]:
with open('/content/drive/My Drive/embed.pkl', 'wb') as f:
    pickle.dump(embeddings,f)